### Sex Ratio of Children of Indian Politicians

In [1]:
import os
import json
import requests
import pandas as pd
import scipy.stats as stats

df_list = []

for filename in os.listdir("data/"):
    print(filename)
    with open(os.path.join("data/", filename), "r") as f:
            data = json.load(f)
    df = pd.DataFrame(data['membersDtoList'])
    df['ls'] = filename[3:-5]
    df_list.append(df)

df = pd.concat(df_list)

ls_17.json
ls_16.json
ls_13.json
ls_14.json
ls_15.json
ls_12.json


In [2]:
df.shape

(3196, 33)

In [3]:
## Filter out NaNs
df_small = df.loc[~ (df['numberOfSons'].isna() | df['numberOfDaughters'].isna())]
df_small.shape

(2983, 33)

In [4]:
sum(df_small['numberOfSons'])/sum(df_small['numberOfDaughters'])

1.0831937465103294

In [5]:
# Filter on ls members with kids
df['total_kids'] = df['numberOfDaughters'] + df['numberOfSons']

df_kids = df[df['total_kids'] > 0]
df_kids['prop_daughter'] = df_kids['numberOfDaughters']/df_kids['total_kids']

print(df_kids['prop_daughter'].mean())

# Perform the t-test
t, p = stats.ttest_1samp(df_kids['prop_daughter'], popmean = 0.4878)

# Print the results
print('t =', t)
print('p =', p)

0.4629706140322832
t = -3.978054398286268
p = 7.130397870998785e-05


/tmp/ipykernel_1079339/259083477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kids['prop_daughter'] = df_kids['numberOfDaughters']/df_kids['total_kids']


In [6]:
df['total_kids'].mean()

2.5015085484411665

In [7]:
df_kids.groupby('ls')['prop_daughter'].mean()

ls
12    0.461634
13    0.464026
14    0.462615
15    0.469007
16    0.469267
17    0.451103
Name: prop_daughter, dtype: float64

#### Let's look at it by party

In [8]:
df_kids_bjp = df_kids[df_kids['partySname'] == 'BJP']
# Perform the t-test
t, p = stats.ttest_1samp(df_kids_bjp['prop_daughter'], popmean = 0.4878)

print(df_kids_bjp['prop_daughter'].mean())
# Print the results
print('t =', t)
print('p =', p)

0.4543823575862864
t = -3.449952944170598
p = 0.0005825870909039917


In [9]:
df_kids_inc = df_kids[df_kids['partySname'] == 'INC']
# Perform the t-test
t, p = stats.ttest_1samp(df_kids_inc['prop_daughter'], popmean = 0.4878)

print(df_kids_inc['prop_daughter'].mean())
# Print the results
print('t =', t)
print('p =', p)

0.4747165532879819
t = -0.9713369547975335
p = 0.3318006131223229


In [18]:
# Create two DataFrames for the two groups
bjp    = df_kids[df_kids['partySname'] == 'BJP']
others = df_kids[~ (df_kids['partySname'] == 'BJP')]

# Perform the t-test
t_statistic, p_value = stats.ttest_ind(bjp['prop_daughter'], others['prop_daughter'])

# Print the results
print(bjp['prop_daughter'].mean())
print(others['prop_daughter'].mean())
print('t-statistic:', t_statistic)
print('p-value:', p_value)

0.4543823575862864
0.4685721064915696
t-statistic: -1.1112914309522917
p-value: 0.2665416543686745


### Sex Ratios

In [11]:
adf = df.groupby('ls')[['numberOfSons', 'numberOfDaughters']].sum().reset_index()
adf['sex_ratio'] = adf['numberOfSons']/adf['numberOfDaughters']
adf

,ls,numberOfSons,numberOfDaughters,sex_ratio
0,12,472.0,444.0,1.063063
1,13,775.0,742.0,1.044474
2,14,755.0,676.0,1.116864
3,15,655.0,630.0,1.039683
4,16,609.0,552.0,1.103261
5,17,614.0,538.0,1.141264


### By Party

In [13]:
adf = df.groupby('partySname')[['numberOfSons', 'numberOfDaughters']].sum().reset_index()
adf['sex_ratio'] = adf['numberOfSons']/adf['numberOfDaughters']
adf.sort_values('numberOfSons', ascending = False)[0:10]

,partySname,numberOfSons,numberOfDaughters,sex_ratio
17,BJP,1591.0,1428.0,1.114146
27,INC,773.0,800.0,0.966250
22,CPI(M),143.0,126.0,1.134921
69,SP,115.0,125.0,0.920000
19,BSP,110.0,79.0,1.392405
73,TDP,91.0,85.0,1.070588
70,SS,89.0,62.0,1.435484
35,JD(U),86.0,64.0,1.343750
16,BJD,81.0,86.0,0.941860
10,AITC,78.0,60.0,1.300000


In [14]:
adf = df.groupby(['ls', 'partySname'])[['numberOfSons', 'numberOfDaughters']].sum().reset_index()
adf['sex_ratio'] = adf['numberOfSons']/adf['numberOfDaughters']
adf.sort_values('numberOfSons', ascending = False)[0:20]

,ls,partySname,numberOfSons,numberOfDaughters,sex_ratio
221,17,BJP,364.0,315.0,1.155556
185,16,BJP,351.0,312.0,1.125000
58,13,BJP,286.0,268.0,1.067164
153,15,INC,231.0,225.0,1.026667
102,14,BJP,218.0,190.0,1.147368
9,12,BJP,202.0,188.0,1.074468
109,14,INC,200.0,194.0,1.030928
145,15,BJP,170.0,155.0,1.096774
65,13,INC,157.0,190.0,0.826316
17,12,INC,93.0,104.0,0.894231
